In [1]:
import cv2
import numpy as np
from PIL import Image

In [2]:
##————————————————————————————————————————————定义函数————————————————————————————————————————————##
##————————————————————————————————————————————————————————————————————————————————————————————————##

# ——————————定义图像显示函数——————————#
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
#——————————定义SIFT特征检测和描述函数——————————#
def SIFTdetectAndDescribe(image):
    # 建立SIFT生成器
    descriptor = cv2.SIFT_create()
    # 检测SIFT特征点，并计算描述子
    (kps, features) = descriptor.detectAndCompute(image, None)
     # 在图像中标出关键点
    img_with_keypoints = cv2.drawKeypoints(image, kps, None)
    show('keypoints', img_with_keypoints)
    
    # 将结果转换成NumPy数组
    kps = np.float32([kp.pt for kp in kps])
    # 返回特征点集，及对应的描述特征
    return (kps, features)

In [4]:
#——————————定义特征点匹配函数——————————#
def matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio = 0.75, reprojThresh = 4.0):

    matcher = cv2.BFMatcher()
    # 使用KNN检测来自A、B图的SIFT特征匹配对，K=2
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
    matches = []
    for m in rawMatches:
        # 当最近距离跟次近距离的比值小于ratio值时，保留此匹配对
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        # 存储两个点在featuresA, featuresB中的索引值
            matches.append((m[0].trainIdx, m[0].queryIdx))
        # 当筛选后的匹配对大于4时，计算视角变换矩阵
    if len(matches) > 4:
        # 获取匹配对的点坐标
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])
        # 计算视角变换矩阵
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
        # 返回结果
        return (matches, H, status)
    # 如果匹配对小于4时，返回None
    return None

In [5]:
#——————————定义特征点可视化函数——————————#
def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
    # 初始化可视化图片，将A、B图左右连接到一起
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB
    # 联合遍历，画出匹配对
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        # 当点对匹配成功时，画到可视化图上
        if s == 1:
            # 画出匹配对
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
    show("drawImg", vis)
    # 返回可视化结果
    return vis

In [6]:
#——————————定义图像拼接函数——————————#
def stitch(imageA,imageB, ratio=0.75, reprojThresh=4.0,showMatches=False):
    #检测A、B图片的SIFT关键特征点，并计算特征描述子
    (kpsA, featuresA) = SIFTdetectAndDescribe(imageA)
    (kpsB, featuresB) = SIFTdetectAndDescribe(imageB)
    # 匹配两张图片的所有特征点，返回匹配结果
    M = matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
    # 如果返回结果为空，没有匹配成功的特征点，退出算法
    if M is None:
        return None
    # 否则，提取匹配结果
    # H是3x3视角变换矩阵      
    (matches, H, status) = M
    # 将图片A进行视角变换，result是变换后图片
    result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
    #show('result', result)

    # 将图片B传入result图片最左端
    result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
    show('result', result)
    # 检测是否需要显示图片匹配
    if showMatches:
        # 生成匹配图片
        vis = drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
        # 返回结果
        return (result, vis,warpImg)
    # 返回匹配结果
    return result


In [7]:
##————————————————————————————————————————————图像拼接主程序————————————————————————————————————————————##
##—————————————————————————————————————————————————————————————————————————————————————————————————————##

#输入待拼接图像right,left
imgr = cv2.imread("1right.png")
imgl = cv2.imread("1left.png")
imgr = cv2.resize(imgr,(0,0),fx=0.4,fy=0.4)
imgl = cv2.resize(imgl,(0,0),fx=0.4,fy=0.4)

#显示待拼接图像
show('right', imgr)
show('left', imgl)

#用SIFT检测right、left图片的关键特征点，并计算特征描述子
kpsr, desr = SIFTdetectAndDescribe(imgr)
kpsl, desl = SIFTdetectAndDescribe(imgl)

#进行特征匹配
# 基于最近邻和随机取样一致性得到一个单应性矩阵
(matches,H,status) = matchKeypoints(kpsr, kpsl, desr, desl)
print(H)
# 绘制匹配结果
drawMatches(imgr, imgl, kpsr, kpsl, matches, status)


# 拼接图像
stitch(imgr, imgl)


[[ 1.00025785e+00  1.38233551e-04  2.75187752e+02]
 [-1.99242121e-04  1.00046383e+00 -4.20742197e-02]
 [-2.71960257e-07  7.65231568e-07  1.00000000e+00]]


array([[[139, 167, 184],
        [142, 170, 187],
        [143, 171, 188],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[141, 169, 186],
        [143, 171, 188],
        [143, 171, 188],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[143, 171, 188],
        [144, 172, 189],
        [143, 171, 188],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[ 18,  55,  71],
        [ 36,  73,  88],
        [ 38,  75,  90],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 49,  88, 103],
        [ 54,  93, 108],
        [ 49,  88, 103],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 20,  59,  74],
        [ 19,  58,  73],
        [ 38,  77,  92],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]